**install이 필요한 라이브러리는 이곳에 작성해주세요!**

In [ ]:
# @title  { display-mode: "code" }
# install dependencies: (use cu101 because colab has CUDA 10.1)
#!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html
! pip install cython pyyaml==5.1

! pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
! gcc --version
# opencv is pre-installed on colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44091 sha256=02c6840ab4d10ceb7fabc80f78787718b57cc4a7b055186beed4b35bb6100a8f
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.1
    Uninstalling PyYAML-6.0.1:
      Successfully uninstalled PyYAML-6.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2023.8.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
distributed 2023.8.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
flax 0.7.2 requires PyYAML>=5.4.1, but you have pyyaml 5.1 which is incompatible.

In [ ]:
# @title  { display-mode: "code" }
%cd /content/
# install detectron2:
! git clone https://github.com/facebookresearch/detectron2
%cd /content/detectron2
! pip install -r requirements.txt
! python setup.py install
! pip install git+https://github.com/facebookresearch/fvcore.git


/content
Cloning into 'detectron2'...
remote: Enumerating objects: 15188, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 15188 (delta 117), reused 133 (delta 79), pack-reused 14979
Receiving objects: 100% (15188/15188), 6.22 MiB | 24.40 MiB/s, done.
Resolving deltas: 100% (10984/10984), done.
/content/detectron2
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        *********************

In [ ]:
# @title  { display-mode: "code" }
%cd /content/
# install detectron resnest
! python -m pip install 'git+https://github.com/zhanghang1989/detectron2-ResNeSt.git'

/content
  Cloning https://github.com/zhanghang1989/detectron2-ResNeSt.git to /tmp/pip-req-build-tgxfpk5y
  Running command git clone --filter=blob:none --quiet https://github.com/zhanghang1989/detectron2-ResNeSt.git /tmp/pip-req-build-tgxfpk5y
  Resolved https://github.com/zhanghang1989/detectron2-ResNeSt.git to commit 54ec87602bad7ee3e398b64fa9af335539c184d0
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.1.1-cp310-cp310-linux_x86_64.whl size=5317029 sha256=90da71a738c33daf9d4ec65580d5c1fb54988c6cbec3f108a19b50b8478771c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-9unxumxd/wheels/5c/a7/27/9d28fc84f5fdb1438e0746227151c5d0df619b85637356ad32
  Created wheel for fvcore: filename=fvcore-0.1.dev200407-py3-none-any.whl size=38738 sha256=18b512e7d3db7536c032e631edcb63400b7d21cbeb850d6e7d12f32b7aa75576
  Stored in directory: /root/.cache/pip/wheels/e3/b3/b5/15b702800c0a8cd66c7a3a4b67f8267acaf43b068b2

In [ ]:
### 밑에 코드 전에 반드시 강제로 런타임 다시 재시작하는 이 코드를 실행하셔야 합니다.
### 알 수 없는 이유로 런타임이 재시작된다는 문구가 뜨는것은 정상입니다.
### 이 코드 실행 후 런타임이 연결되면 아래 코드부터 차례대로 실행시키면 됩니다.

! kill -9 $(ps -A | grep python | awk '{print $1}')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import csv
import json
import sys
import torch
import torchvision.models as models
import torch.nn as nn
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.pyplot as plt
import shutil
#import rembg
import joblib
from urllib.parse import urlparse
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from google.colab.patches import cv2_imshow
import tensorflow as tf

# 진행상황을 파악하는 데에 도움이 되며 ram을 정리하는 라이브러리도 아래에 임포트합니다.
from tqdm import tqdm
import time
import gc
import psutil

# detectron2의 의존성 라이브러리입니다.
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader

import warnings
warnings.filterwarnings('ignore')

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")
    device = torch.device('cuda')
else:
    print("Using CPU")
    device = torch.device('cpu')

Using CUDA


In [ ]:
# getpath 함수 설명
# 파라미터 : path(기본 경로에 join으로 찾을 경로주소)
# 예를 들어 path 파라미터로 상품매핑TrainTestFolder/train 을 입력할 수 있습니다.

original_path="/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023"

def getpath(path) :
    result_path = os.path.join(original_path, path)
    return result_path

afterpath = 'jsonnew_1.json'
jsonnewfile_path = os.path.join(getpath('Json/ForDetectron2'), afterpath)

# 새로운 이미지로 폴더 변경하기 위해 기존이미지 삭제
if os.path.exists(getpath('상품매핑TrainTestFolder/train')):
  shutil.rmtree(getpath('상품매핑TrainTestFolder/train'))
  shutil.rmtree(getpath('상품매핑TrainTestFolder/test'))

In [ ]:
# py로 작성된 파일을 불러오는 코드입니다.
# Functions 폴더에 있습니다.

sys.path.append(getpath('Functions'))
from Splitandmakedir import split_data, make_dir, putindata, loadinzip, removeinFolder
from MakeFile import makejsonfile, splitjson, get_dicts
from SaveModelOutputs import copymodelpth, modeloutputs, showresult, showpercentresult, addresultimg, load_image, save_image, makecropimg
from ModelClass import ResNet, GoogLeNet, ResGoogLeNet, MyDataset, train, test

**현재는 예선에서 url형태로 받는다는 가정하에 아래처럼 작성했습니다. 따라서 urllist에 각 url을 넣어주시면 됩니다. 만약 에러가 나거나 url이 아닌 다운받는 형태이면 수작업으로 해당 폴더에 넣어주시면 되며 putindata 코드는 주석처리 해주시면 됩니다.**

**그리고 NewImage/trainimage 경로로 담겨질 train 이미지를 augmentation하신다음에 폴더 하나에 zip하지 마시고 담으신 다음에 augmentation 폴더 내부에 옮겨주시면 됩니다.**

In [ ]:
start = time.time()
#'url', 'name'+',' + getpath('NewImage/trainimage')형태, #'url', 'name'+',' + getpath('NewImage/testimage')형태
# 예선 시에 새로 받는 이미지를 전처리 이미지의 train 경로로 복사하는 함수입니다.
# 현재는 NewImage/trainimage, NewImage/testimage 폴더가 비어 있고, 예선에서 url형태로 받는다는 가정하에 아래처럼 작성했습니다.
# 만약 url이 아니라 다운받는 형태이면 수작업으로 해당 폴더에 6장 넣어주시면 되며 아래 코드는 주석처리 해주시면 됩니다.

map = [
    {'': '' + ',' + getpath('NewImage/trainimage')},#train image,
    {'': '' + ',' + getpath('NewImage/trainimage')},#train image,
    {'': '' + ',' + getpath('NewImage/trainimage')},#train image,
    {'': '' + ',' + getpath('NewImage/trainimage')},#train image,
    {'': '' + ',' + getpath('NewImage/trainimage')},#train image,
    {'': '' + ',' + getpath('NewImage/testimage')},#validation image,
]


# putindata 함수 설명
# 파라미터 :
# getdir는 새로 받는 상품이 담겨질 경로로 image_preprocessing/NewImage 또는 'validation/set11'입니다.
# imgurl는 새로 받는 상품을 웹url로 받는다고 가정하였습니다. url을 해당 파라미터에 입력해주세요!
# imgname는 새로 받는 상품의 이미지 이름입니다.

for item in map:
  for key, value in item.items():
    split_value = value.split(',')
    if split_value[0] == '':
      print('폴더내에 어떤 이미지도 존재하지 않습니다.')
    else:
      putindata(
          getdir=split_value[1],
          imgurl=key,
          imgname=split_value[0]
      )


# 총 소요시간 출력
end = time.time()
runningtime = end - start
print(f"\nRunning Time: {runningtime:.2f}초")

폴더내에 어떤 이미지도 존재하지 않습니다.
폴더내에 어떤 이미지도 존재하지 않습니다.
폴더내에 어떤 이미지도 존재하지 않습니다.
폴더내에 어떤 이미지도 존재하지 않습니다.
폴더내에 어떤 이미지도 존재하지 않습니다.
폴더내에 어떤 이미지도 존재하지 않습니다.

Running Time: 0.00초


In [ ]:
def TotalTrainer():
  # image_preprocessing_code.ipynb 내부의 로직을 이곳에 작성하였습니다.
  # 이 곳의 코드를 실행시키면 해당 파일을 따로 런타임실행하지 않아도 됩니다.
  start = time.time()

  # split_data 함수 설명(image_extractionpart_pretrainedmodel.ipynb에서 결과를 보시면 한 폴더당 20초 내외이므로 시간에 크게 영향을 미치지 않습니다.)
  # train/test이미지를 랜덤으로 0.7/0.3의 비로 나누어 담는 함수입니다.
  # NewImage/trainimage 경로의 train 이미지를 augmentation하신다음에 폴더 하나에 zip하지 마시고 담으신 다음에 augmentation 폴더 내부에 옮겨주시면 됩니다.
  # 파라미터 :
  # src_dir는 상품이 담겨진 기본 경로인 /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/augmentation
  # train_dir는 전처리 될 train 상품이 담기는 기본 경로인 /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑TrainTestFolder/train
  # test_dir는 전처리 될 test 상품이 담기는 기본 경로인 /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑TrainTestFolder/test
  # ratio는 split 비율인데 이 프로젝트에서는 0.7/0.3으로 설정하고 진행합니다.

  isexits = True
  while isexits:
    if os.path.exists(getpath('상품매핑TrainTestFolder/train')):
      isexits = True
      print('아직 진행중인 작업처리중이므로 잠시 기다려주십시오.')
    else:
      zippath = loadinzip(src_dir=getpath('augmentation'),)
      for path in zippath:
        split_data(
            src_dir=getpath(path.split('/')[5] + '/' + path.split('/')[6]),
            train_dir=getpath('상품매핑TrainTestFolder/train'),
            test_dir=getpath('상품매핑TrainTestFolder/test'),
            ratio=0.7)
      isexits = False

  # train/test를 데이터셋에 등록하고 로드하는 방법입니다.

  dataset_train = MyDataset(labeledimg_dir=getpath('상품매핑TrainTestFolder/train'))
  dataloader_train = DataLoader(dataset=dataset_train, batch_size=64)
  dataset_test = MyDataset(labeledimg_dir=getpath('상품매핑TrainTestFolder/test'))
  dataloader_test = DataLoader(dataset=dataset_test, batch_size=64)


  # imagebyimage 함수 설명
  # 전처리하는 함수였지만 위에서 augmentation 이미지들로 대체함에 따라 주석처리합니다.
  # 파라미터 :
  # dataurl는 새로운 이미지가 담긴 폴더 경로인 /content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/image_preprocessing/NewImage

  #allimagepreprocessing(
  #    dataurl=getpath('상품매핑TrainTestFolder/train'),
  #    trainortest = 'train')
  #allimagepreprocessing(
  #    dataurl=getpath('상품매핑TrainTestFolder/test'),
  #    trainortest = 'test')

  #imagebyimage(
  #    dataurl=getpath('NewImage/trainimage'),)


  # 모델을 학습하는 코드입니다. 예선 당일 받게 되는 이미지를 추가로 학습하여야 하기에 실행하셔야 합니다.
  # Resnet으로 수정함.
  # best model을 저장할 변수를 선언합니다.
  best_model = None
  best_acc = 0
  if not os.path.exists(getpath('backbonefolder/epoch30withrn/bestmodel')):
    os.makedirs(getpath('backbonefolder/epoch30withrn/bestmodel'))
  else:
    None
  if not os.path.exists(getpath('backbonefolder/epoch30withrn/bestmodel/backbonemodel-ResNet.h5')):
    backbone_model_path = getpath('backbonefolder/epoch30withrn/backbonemodel-ResNet.h5')
  else:
    backbone_model_path = getpath('backbonefolder/epoch30withrn/bestmodel/backbonemodel-ResNet.h5')
  model = joblib.load(backbone_model_path)

  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters())
  for epoch in range(17):
      tr_time, train_acc = train(model, dataloader_train, criterion, optimizer)
      te_time, test_acc = test(model, dataloader_test)
      print(f"\nEpoch {epoch+1}, Train Accuracy: {train_acc:.2f}%, Train Time: {tr_time:.2f}초, Test Accuracy: {test_acc:.2f}%, Test Time: {te_time:.2f}초")
  # test accuracy가 best accuracy보다 높으면 best model을 업데이트합니다.
  if test_acc > best_acc:
      best_acc = test_acc
      best_model = model

  # 모델을 저장합니다.
  new_backbone_model_path = getpath('backbonefolder/epoch30withrn/bestmodel/backbonemodel-ResNet.h5')
  joblib.dump(best_model, new_backbone_model_path)
  #torch.save(model.state_dict(), path) # 모델을 저장합니다.


  # 총 소요시간 출력
  end = time.time()
  runningtime = end - start
  print(f"\nRunning Time: {runningtime:.2f}초")


In [ ]:
outputlist = []
outlist = []
img_path = None
dataset_dicts = get_dicts(original_path, jsonnewfile_path)
def iou(box1, box2):
    def _is_box_intersect(box1, box2):
        if (
            abs(box1[0] - box2[0]) < box1[2] + box2[2]
            and abs(box1[1] - box2[1]) < box1[3] + box2[3]
        ):
            return True
        else:
            return False

    def _get_area(box):			# area of box n.
        return box[2] * box[3]

    def _get_intersection_area(box1, box2):
    # intersection area
        return abs(max(box1[0], box2[0]) - min(box1[0] + box1[2], box2[0] + box2[2])) * abs(
            max(box1[1], box2[1]) - min(box1[1] + box1[3], box2[1] + box2[3])
        )
    def _get_union_area(box1, box2, inter_area=None):
        area_a = _get_area(box1)
        area_b = _get_area(box2)
        if inter_area is None:
            inter_area = _get_intersection_area(box1, box2)

        return float(area_a + area_b - inter_area)

    # if boxes do not intersect
    if _is_box_intersect(box1, box2) is False:
        return 0

    inter_area = _get_intersection_area(box1, box2)
    union = _get_union_area(box1, box2, inter_area=inter_area)

    # intersection over union
    iou = inter_area / union
    if iou < 0:
        iou = 0
    assert iou >= 0, f"Measure is wrong! : IoU Value is [{iou}]."
    return iou


def nms(boxes, iou_thres):
    elems = np.array(boxes)
    iou_list = []
    # print("\nBefore Arrange")
    #print(elems)

    sorted_index = np.argsort(elems[:, -1])[::-1]
    sorted_boxes = elems[sorted_index]

    # print("\nAfter Arrange")
    #print(sorted_boxes)

    answer = [True for x in range(sorted_boxes.shape[0])]
    # print("\nBefore NMS Answer :", answer)

    for i in range(sorted_boxes.shape[0]):
        if answer[i] is False:
            continue
        for j in range(sorted_boxes.shape[0]):
            iou_val = iou(sorted_boxes[i], sorted_boxes[j])
            #print(f"{i} vs {j} = iou {round(iou_val, 3)}")
            if iou_val >= iou_thres and int(iou_val) != 1:
                answer[j] = False
                # print(f"Index {j} is False.")
            iou_list.append(iou_val)

    # print("\nAfter NMS Answer :", answer)

    return answer, sorted_boxes, sorted_index, iou_list
def TotalTester():
  # image_preprocessing_code.ipynb 내부의 로직을 이곳에 작성하였습니다.
  # 이 곳의 코드를 실행시키면 해당 파일을 따로 런타임실행하지 않아도 됩니다.
  start = time.time()

  #splitjson 함수와 makejsonfile 함수를 거쳐서 json1_10과 jsonnew_1으로 명명된 json파일이 존재하지 않으면 생성하고 경로를 리스트에 담는 방법입니다.
  json1_10 = splitjson(99)
  jsonnew_1 = splitjson(num=99, isnewfile=True, filename=jsonnewfile_path)

  makejsonfile(json1_10, 'json1_10', 0)
  makejsonfile(jsonnew_1, 'jsonnew_1', 0)

  json1_10=getpath('Json/ForDetectron2/json1_10.json')
  jsonnew_1=jsonnewfile_path
  pathlist = [json1_10, jsonnew_1]


  # 훈련 데이터셋 경로를 아래와 같이 설정합니다.
  if 'cj_train' in DatasetCatalog.list():
      del DatasetCatalog._REGISTERED['cj_train']
  if 'cj_val' in DatasetCatalog.list():
      del DatasetCatalog._REGISTERED['cj_val']

  for d in ["train", "val"]:
    if d == 'train':
      path = pathlist[0]
    else:
      path = pathlist[1]
    DatasetCatalog.register("cj_" + d, lambda d=d: get_dicts(original_path, path))
    MetadataCatalog.get("cj_" + d).set(thing_classes=["boxes"])
  cj_train_metadata = MetadataCatalog.get("cj_train")
  cj_val_metadata = MetadataCatalog.get("cj_val")


  # detectron2를 미리 1_10 json file로 학습하였기 때문에 아래의 test하는 코드만 실행하시면 됩니다.
  # 만약에 다시 학습을 시켜야한다면 기존의 detectron2_final.ipynb 파일의 마크다운으로 처리한 학습용 코드를 먼저 추가하시고 실행하시면 됩니다.
  cfg = get_cfg()
  cfg.MODEL.DEVICE = "cpu"
  cfg.DATASETS.TEST = ('cj_val',)
  cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
  cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (boxes).
  cfg.MODEL.WEIGHTS = os.path.join(getpath('backbonefolder'), 'detectron2model.pth') # 새로 복사된 경로
  #cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # 원래의 경로
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.39  # set a custom testing threshold

  predictor = DefaultPredictor(cfg)
  outputlist.clear()
  outlist.clear()
  for d in dataset_dicts:
      img_path = d["file_name"]
      im = cv2.imread(img_path)
      outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
      v = Visualizer(im[:, :, ::-1],
                      metadata=cj_val_metadata,
                      scale=0.5,
                      )
      outputlist.append(outputs)
      out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      outlist.append(out)

  txtlist = showresult(outputlist)
  df = pd.DataFrame(columns=['Index','BBOX'])
  for idx, filetxt in enumerate(txtlist):
    df.loc[idx] = [idx, filetxt]
  df
  # bbox 데이터프레임을 csv 파일로 저장
  modeloutputs(df, 'bbox_dataframe.csv', 0)

  txtlist2 = showpercentresult(outputlist)
  df2 = pd.DataFrame(columns=['Index','Percentage'])
  for idx, filetxt in enumerate(txtlist2):
    df2.loc[idx] = [idx, filetxt]
  df2
  # Percentage 데이터프레임을 csv 파일로 저장
  modeloutputs(df2, 'percentage_dataframe.csv', 0)

  #nms
  outlists = nms(txtlist[0], iou_thres = 0.3)
  # 결과 이미지와 bbox를 보여줍니다.
  addresultimg(lst=outlists, path=img_path)
  # crop img folder 코드
  # 새로운 이미지로 폴더 변경하기 위해 기존이미지 삭제
  if os.path.exists(getpath('상품매핑PreprocessedFolder')):
    shutil.rmtree(getpath('상품매핑PreprocessedFolder'))
  if not os.path.exists(getpath('상품매핑PreprocessedFolder')):
    os.makedirs(getpath('상품매핑PreprocessedFolder'))
  makecropimg(lst=outlists, path=img_path)



  # 총 소요시간 출력
  end = time.time()
  runningtime = end - start
  print(f"\nRunning Time: {runningtime:.2f}초")

  return outlists

In [ ]:
# init
object_detection_model_path = getpath('OutputModel/ForDetectron2/detectron2model.pth')
backbone_model_path = getpath('backbonefolder/epoch30withrn/bestmodel/backbonemodel-ResNet.h5')
validation_img_path = getpath('NewImage/testimage')
crop_img_path = getpath('상품매핑PreprocessedFolder')
bbox_csv_save_path = getpath('OutputModel/ForDetectron2/bbox_dataframe.csv')
percentage_csv_save_path = getpath('OutputModel/ForDetectron2/percentage_dataframe.csv')
res_img_path = getpath('Result/outputimg.jpg')
res_csv_path = getpath('Result/output.csv')
res_txt_path = getpath('Result/output.txt')
train_path = getpath('상품매핑TrainTestFolder/train')

bbox_df = pd.read_csv(bbox_csv_save_path, encoding="utf-8")
bbox_df = json.loads(bbox_df['BBOX'][0])
object_detection_model = torch.load(object_detection_model_path)
backbone_model = joblib.load(backbone_model_path)
dataset_dicts = get_dicts(original_path, jsonnewfile_path)
dataset_res = MyDataset(labeledimg_dir=crop_img_path)
dataloader_res = DataLoader(dataset=dataset_res, batch_size=1)

In [ ]:
#function call
def predictfunc(model, dataloader):
  predictedlist = []
  #iou_lr = []
  cnt = 0
  start = time.time()
  model.eval() # 모델을 평가 모드로 설정합니다.
  pbar = tqdm(dataloader,
            total = len(dataloader), ## 전체 진행수
            desc = 'Description Testing', ## 진행률 앞쪽 출력 문장
            ncols = 100, ## 진행률 출력 폭 조절
            ascii = ' =', ## 바 모양, 첫 번째 문자는 공백이어야 작동
            leave = True, ## True 반복문 완료시 진행률 출력 남김. False 남기지 않음.
           )
  with torch.no_grad(): # 가중치 업데이트와 메모리 사용량을 방지합니다.
      for imgs in pbar:
          imgs = imgs.to(device) # 이미지와 라벨을 해당 장치로 옮깁니다.
          outputs = model(imgs) # 모델에 이미지를 입력하여 출력값을 얻습니다.
          _, predicted = torch.max(outputs.data, 1) # 출력값에서 가장 큰 값의 인덱스를 예측값으로 사용합니다.
          file_list = os.listdir(train_path)
          if predicted.item() > len(file_list):
            cnt += 1
          else:
            filename = file_list[predicted.item()]
            label = filename.split('_')[1][:2 if len(filename.split('_')[1].split('.')[0]) == 4 else 3]
            predictedlist.append(label)
            #iou_lr.append(ioulist[cnt])
            cnt += 1
  return predictedlist #iou_lr
def getlabelnamelist() :
  # 전체 라벨과 상응하는 상품명을 담은 코드입니다.
  labelnamelist = []
  for labelname in set(os.listdir(getpath('상품매핑'))):
    labelnum = labelname.split(')')[0]
    label = {labelnum: labelname.split(') ')[1].split('.')[0]}
    labelnamelist.append(label)
  labelnamelist = [dict(t) for t in {tuple(d.items()) for d in labelnamelist}]
  return labelnamelist
def iou2(box1, box2):
    # box1과 box2의 좌표를 각각 변수에 할당
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2
    # box1과 box2의 교차 영역의 좌표를 계산
    x5 = max(x1, x3)
    y5 = max(y1, y3)
    x6 = min(x2, x4)
    y6 = min(y2, y4)
    # 교차 영역의 넓이를 계산
    inter_area = max(0, x6 - x5) * max(0, y6 - y5)
    # box1과 box2의 합집합 영역의 넓이를 계산
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x4 - x3) * (y4 - y3)
    union_area = box1_area + box2_area - inter_area
    # iou값을 계산하고 반환
    iou_value = inter_area / union_area
    return iou_value

def makepredicts(itemlist, ioulist_real, ioulist_pred, ioupercent):

  #원본 이미지에서 라벨을 얻어오는 코드입니다.
  labelnamelist = getlabelnamelist()

  # result iou save
  # outlist에서 결과가 true인 리스트의 인덱스를 얻는 부분 고민...

  df = pd.DataFrame(columns=['상품명', 'IOU'])
  count = 0
  for labels in itemlist:
    for ln in labelnamelist:
      if str(labels) in ln:
        # IOU는 임의로 모두 0.5로 설정하였습니다. IOU가 아직 불완전해서.. 추후에 주석처리된 코드로 넣으면 됩니다.
        #df.loc[count] = [ln[str(labels)], ioulist[count]]
        df.loc[count] = [ln[str(labels)], ioupercent[count]]
        count += 1
  df.to_csv(res_csv_path, index = False, encoding='utf8')
  with open(res_txt_path, 'w+') as my_output_file:
    with open(res_csv_path, 'r') as my_input_file:
      # CSV 파일의 각 행을 공백으로 구분하여 TXT 파일에 쓰기
      [my_output_file.write(' '.join(row) + '\n') for row in csv.reader(my_input_file)]
  os.remove(res_csv_path)

  for d in dataset_dicts:
    img = load_image(d["file_name"])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for idx in range(len(ioulist_real)):
      # 4개의 점의 좌표를 정합니다
      iou1_x1, iou1_y1, iou1_x2, iou1_y2 = ioulist_real[idx][0], ioulist_real[idx][1], ioulist_real[idx][2], ioulist_real[idx][3]
      p1 = (int(iou1_x1), int(iou1_y1))
      p3 = (int(iou1_x2), int(iou1_y2))
      cv2.rectangle(img, p1, p3, (255, 0, 0), 2)
    for idx in range(len(ioulist_pred)):
      # 4개의 점의 좌표를 정합니다
      iou2_x1, iou2_y1, iou2_x2, iou2_y2 = ioulist_pred[idx][0], ioulist_pred[idx][1], ioulist_pred[idx][2], ioulist_pred[idx][3]
      p1 = (int(iou2_x1), int(iou2_y1))
      p3 = (int(iou2_x2), int(iou2_y2))
      cv2.rectangle(img, p1, p3, (255, 255, 0), 2)
    # result image save
    save_image(img, res_img_path)
def showResults(outlists):
  #main loop
  start = time.time()
  predictitems = []
  ioulist_real = []
  ioulist_pred = []
  iou_percent = []
  true_indices = []
  False_indices = []
  new_pred = []
  new_real = []
  for i, value in enumerate(outlists[3]): # 리스트의 인덱스와 값에 대해 반복문 실행
    if value == False: # 값이 True이면
      False_indices.append(i) # 인덱스를 리스트에 추가
    else: # 값이 True이면
      true_indices.append(i) # 인덱스를 리스트에 추가
  ioulist_pred = [outlists[1][i] for i in true_indices if i < len(outlists[1])]
  False_indices.clear()
  true_indices.clear()
  #result txt 출력하는 함수
  dataset_dicts = get_dicts(original_path, jsonnewfile_path)
  for d in dataset_dicts:
    for i in range(len(d['annotations'])):
      ioulist_real.append(d["annotations"][i]['bbox'])
  for pred_box in ioulist_pred: # ioulist_pred의 각 원소에 대해 반복문 실행
    for real_box in ioulist_real: # ioulist_real의 각 원소에 대해 반복문 실행
      iou_value = iou(pred_box, real_box) # 두 박스의 iou값을 계산
      if iou_value >= 0.5 and iou_value < 1:
        new_pred.append(pred_box)
        new_real.append(real_box)
        iou_percent.append(iou_value)
  #new_pred = list(set(new_pred)) # new_pred를 set으로 변환하여 중복된 원소를 제거하고, 다시 list로 변환
  #new_real = list(set(new_real)) # new_real을 set으로 변환하여 중복된 원소를 제거하고, 다시 list로 변환
  new_pred = list({tuple(box): None for box in new_pred}.keys()) # new_pred의 각 원소를 튜플로 변환하여 딕셔너리의 키로 사용하고, 딕셔너리의 키들을 리스트로 변환
  new_real = list({tuple(box): None for box in new_real}.keys()) # new_real의 각 원소를 튜플로 변환하여 딕셔너리의 키로 사용하고, 딕셔너리의 키들을 리스트로 변환

  predicted = predictfunc(backbone_model, dataloader_res)
  predictitems = predicted
  makepredicts(predictitems, new_real, new_pred, iou_percent)
  # 총 소요시간 출력
  end = time.time()
  runningtime = end - start
  print(f"\nRunning Time: {runningtime:.2f}초")

In [ ]:
# 예선 시에 Training 시간에는 이 Trainer를 돌리시면 됩니다.
# 전처리된 TrainTestFolder 이미지로 시간이 가장 빠른 resnet backbone 학습에 30분 이내 소요.

TotalTrainer()

Train : 417
Test : 179


Description Shutil TestFolder: 100%|==============================| 179/179 [00:02<00:00, 77.23it/s]


Train : 417
Test : 179


Description Shutil TestFolder: 100%|==============================| 179/179 [00:02<00:00, 64.20it/s]


Train : 417
Test : 179


...


Epoch 15, Train Accuracy: 1.00%, Train Time: 29.77초, Test Accuracy: 0.99%, Test Time: 23.43초


Description Testing: 100%|============================================| 9/9 [00:22<00:00,  2.45s/it]



Epoch 16, Train Accuracy: 1.00%, Train Time: 29.94초, Test Accuracy: 0.99%, Test Time: 22.09초


Description Testing: 100%|============================================| 9/9 [00:23<00:00,  2.61s/it]



Epoch 17, Train Accuracy: 1.00%, Train Time: 29.85초, Test Accuracy: 0.99%, Test Time: 23.53초

Running Time: 963.61초


In [ ]:
# 예선 시에 Test 시간에는 이 Tester를 돌리시면 됩니다.
# 현재 cart_1001 validation image를 임의로 set11 폴더에 cart_1101로 복사 후 진행한 결과입니다.
# 이미지 하나에 30초 이내로 소요됨을 알 수 있습니다.
# 수정된 코드 이미지

outlists=TotalTester()

In [ ]:
showResults(outlists)